In [316]:
%run '/Users/zhaowenliu/Documents/Tooklits/pyuvvis/examples/Notebooks/NBCONFIG.ipynb'

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [317]:
from IPython.html.widgets import (
    FlexBox, VBox, HBox, HTML, Box, RadioButtons,
    FloatText, Dropdown, Checkbox, Image, IntSlider, Button, Text, FloatSlider, IntText, ContainerWidget
)
from IPython.utils.traitlets import (
    link, Unicode, Float, Int, Enum, Bool, Instance
)

%matplotlib inline

import matplotlib.pyplot as plt

from collections import OrderedDict

from matplotlib.pyplot import savefig

Use [`OrderedDict`](https://docs.python.org/2/library/collections.html#collections.OrderedDict) for predictable display of key-value pairs.

In [318]:
%%html
<style>
/*
    This contents of this would go in a separate CSS file.

    Note the namespacing: this is important for two reasons.
    1) doesn't pollute the global namespace
    2) is _more specific_ than the base styles.
*/

.widget-area .spectroscopy .panel-body{
    padding: 0;
}
.widget-area .spectroscopy .widget-numeric-text{
    width: 5em;
}
.widget-area .spectroscopy .widget-box.start{
    margin-left: 0;
}
.widget-area .spectroscopy .widget-hslider{
    width: 12em;
}
.widget-area .spectroscopy .widget-text{
    width: 13em;
}
    
</style>

In [319]:
class PanelTitle(HTML):
    def __init__(self, *args, **kwargs):
        super(PanelTitle, self).__init__(*args, **kwargs)
        self._dom_classes += ("panel-heading panel-title",)

class PanelBody(Box):
    def __init__(self, *args, **kwargs):
        super(PanelBody, self).__init__(*args, **kwargs)
        self._dom_classes += ("panel-body",)

class ControlPanel(Box):
    # A set of related controls, with an optional title, in a box (provided by CSS)
    def __init__(self, title=None, *args, **kwargs):
        super(ControlPanel, self).__init__(*args, **kwargs)
        self._dom_classes += ("panel panel-info",)
        
        # add an option title widget
        if title is not None:
            
            self.children = [
                PanelTitle(value=title),
                PanelBody(children=self.children)
            ]
        
class ToolBar(Button):
    def __init__(self, title=None, *args, **kwargs):
        super(ToolBar, self).__init__(*args, **kwargs)
        self._dom_classes += ("btn-success",)

In [320]:
import pyuvvis
from pyuvvis.data import aunps_glass
from pyuvvis.core.spectra import _normdic as NUdic
ts=aunps_glass()

SUdic = ts.specunits()
VUdic = ts.varunits()
NUdic = NUdic

print SUdic
print VUdic
print NUdic

Colordic = ["blue","green","red","cyan","magenta","yellow","black","white"]
Colormaps = sorted(m for m in plt.cm.datad if not m.endswith("_r"))

{'um-1': 'inverse microns', 'nm': 'nanometers', 'cm': 'centimeters', 'f': 'hertz', 'k': 'inverse meters', 'm': 'meters', 'cm-1': 'inverse centimeters', 'um': 'microns', 'w': 'radians per second', None: 'No unit', 'ev': 'electron volts'}
{'dti': 'timestamp', 'd': 'days', 'h': 'hours', 'm': 'minutes', 'us': 'microseconds', 's': 'seconds', 'ms': 'milliseconds', 'intvl': 'interval', None: 'No unit', 'ns': 'nanoseconds'}
{'a': 'Absorbance (base 10)', 'r': 'Inverse Transmittance (1/T)', 't': 'Transmittance', 'a(ln)': 'Absorbance (base e)', '%t': '(%)Transmittance', None: 'No Normalization'}


In [321]:
def html_figure(fig, title="Figure"):
    
    from IPython.core.pylabtools import print_figure
    import base64

    fdata64 = base64.b64encode(print_figure(fig))
    html_tpl = '<img alt="{title}" src="data:image/png;base64,{fdata64}">'
    return html_tpl.format(**locals())

In [322]:
import re
from datetime import datetime
from pyuvvis.plotting.advanced_plots import PLOTPARSER
from pyuvvis import TimeSpectra

class Spectrogram(HTML, Box):
    """
    A notional "complex widget" that knows how to redraw itself when key properties change.
    """
    DONT_DRAW = re.compile(r'^(_.+|value|keys|comm|children|visible|parent|log|config|msg_throttle)$')
    
    # Lookup tables: this would be a nice place to add i18n, perhaps    
    SPECUNITS = OrderedDict(sorted(SUdic.items(), key = lambda x: x[0]))
    SPECUNITS_REV = OrderedDict((v,k) for k,v in SPECUNITS.items())
    VARUNITS = OrderedDict(sorted(VUdic.items(), key = lambda x: x[0]))
    VARUNITS_REV = OrderedDict((v,k) for k,v in VARUNITS.items())
    NORMUNITS = OrderedDict(NUdic)
    NORMUNITS_REV = OrderedDict((v,k) for k,v in NORMUNITS.items())
    COLORS = OrderedDict([(x,x) for x in Colordic])    
    COLORMAPS = Colormaps
    
    
    # pass-through traitlets
    load_TS = Bool(False,sync=True)
    load_file = Bool(True,sync=True) #
    TS_name = Unicode("ts", sync=True)
    file_name = Unicode("aunps_glass", sync=True)
    
    #ts = Instance(TimeSpectra)

    # Plotting Traits
    colorbar = Bool(False, sync=True)
    autoupdate = Bool(True, sync=True)
    colormap = Enum(COLORMAPS,sync=True)
    plot_color = Enum(COLORS.values(), default_value=COLORS.values()[0], sync=True)
    kind = Enum(PLOTPARSER.keys(), default_value = 'spec', sync=True)

    # Sampling Traits (defaults depend on time spectra)
    spec_unit = Enum(SPECUNITS.values(), sync=True)
    var_unit = Enum(VARUNITS.values(), sync=True)
    norm_unit = Enum(NORMUNITS.values(),  sync=True)
    sample_axis = Bool(False, sync=True)
    sample_range = Bool(True, sync=True)

    axis_rows_1 = Int(0,sync=True)
    axis_rows_2 = Int(100,sync=True)
    axis_cols_1 = Int(0, sync=True)
    axis_cols_2 = Int(10, sync=True)
    dataset_spacing = Int(1, sync=True)
    
    
    slicing_start = Float(sync=True)
    slicing_end = Float(sync=True)
    
    save_ts = Bool(False,sync=True)
    save_ts_as = Unicode('Test',sync=True)

    
    def __init__(self, *args, **kwargs):
        """
        Creates a spectrogram
        """
        default_ts = kwargs.pop('default_ts', 'aunps_glass')
        self.ts = getattr(pyuvvis.data, default_ts)()
        self.ts_modified = self.ts

        # Super will call default values
        super(Spectrogram, self).__init__(*args, **kwargs)

        
        # Do not change, put last
        self.on_trait_change(lambda name, old, new: self.draw(name, old, new))
        self._dom_classes += ("col-xs-9",)
        self.draw()
    

            
    # DEFAULTS
    # --------
    def _spec_unit_default(self):
        return self.ts.full_specunit
    
    def _var_unit_default(self):
        print 'in default var'
        return self.ts.full_varunit
    
    def _norm_unit_default(self):
        return self.ts.full_norm
        
    def _slicing_start_default(self): 
        print 'im defaulting to self.ts.index[0]'
        return self.ts.index[0]
   
    def _slicing_end_default(self):
        print 'in slicing end'
        return self.ts.index[-1]
        
    def _colormap_default(self):
        return 'jet'
    

    # Events
    # ------
    def _file_name_changed(self, name, old, new):
        try:
            ts = getattr(pyuvvis.data, new)()
        except AttributeError:
            pass
        else:
            self.ts = ts
            
    def save_to_ns(self):
        get_ipython().user_ns[self.save_ts_as]=self.ts_modified
        print "it has been saved"
    
    def draw(self, name=None, old=None, new=None):
        if name is not None and self.DONT_DRAW.match(name):
            return
        
        value = ''

        # Better way would be a decorator or something that only goes into draw if not autoupdate
        if self.autoupdate:
        
            # Make figure with random data. Replace with real application here
            ts_sample = self.ts.as_specunit(self.SPECUNITS_REV[self.spec_unit]).as_varunit(self.VARUNITS_REV[self.var_unit])
            
            if self.sample_range:
                ts_sample = ts_sample.nearby[self.slicing_start:self.slicing_end,::self.dataset_spacing]
            elif self.sample_axis:
                ts_sample = ts_sample.iloc[self.axis_rows_1:self.axis_rows_2, self.axis_cols_1:self.axis_cols_2]
        
                
            # Generate new figure object
            f = plt.figure()
            if PLOTPARSER.is_3d(self.kind):
                projection = '3d'
            else:
                projection = None
            ax = f.add_subplot(111, projection=projection)

            ts_sample.plot(ax=ax, 
                  fig=f, 
                  kind=self.kind, 
                  norm=self.NORMUNITS_REV[self.norm_unit],
                  colormap=self.colormap,
                  cbar=self.colorbar
                    )
            plt.close(f)
            
            self.fig_old = f
            self.ts_modified = ts_sample
            value = html_figure(f)
            
        else:
            value = html_figure(self.fig_old)
            
        
        #print value
        
        # WATCH THIS! setting value to an html string with the figure data
        # is what will show the plot in the final render:
                
        if name is None:
            value += '<div class="alert alert-info">redraw forced at %s!</div>' % (
                datetime.now().isoformat(' ')
            )
        
        #value += "\n".join([
        #    '<p><span class="label label-%s">%s</span> %s</p>' % (
        #        'success' if traitlet == name else 'default',
        #        traitlet,
        #        getattr(self, traitlet)
        #    )
        #    for traitlet in sorted(self.trait_names())
        #    if not self.DONT_DRAW.match(traitlet)
        #])
        self.value = value

In [323]:
class Spectroscopy(Box):
    """
    An example GUI for a spectroscopy application.
    
    Note that `self.graph` is the owner of all of the "real" data, while this
    class handles creating all of the GUI controls and links. This ensures
    that the Graph itself remains embeddable and rem
    """
    
    
    def __init__(self, graph=None, graph_config=None, *args, **kwargs):
        self.graph = graph or Spectrogram(**(graph_config or {}))
        # Create a GUI
        kwargs["orientation"] = 'horizontal'
        kwargs["children"] = [
            HBox(children=[self.graph,self._toolbars()]),
            self._controls()
        ]
        super(Spectroscopy, self).__init__(*args, **kwargs)
        self._dom_classes += ("spectroscopy row",)
    
    def _controls(self):
        panels = HBox(children=[
            VBox(children=[
                self._loadfile(),
                self._units(),
            ]),
            self._1dplot(),
            self._sampling()
        ],
        _dom_classes=["col-xs-3"])
        
        return panels
    
    def _loadfile(self):
        # create correlation controls. NOTE: should only be called once.
        loadbutton = Button(description = 'Load')
        
        loaddata = Checkbox(description="Load TS")
        link((self.graph, "load_TS"), (loaddata, "value"))
        tsname = Text(description = "TS Name")
        link((self.graph, "TS_name"), (tsname, "value"))
        tsbox = VBox(children=[tsname,loadbutton])
        link((self.graph, "load_TS"), (tsbox, "visible"))
        
        loadfile = Checkbox(description="Load File")
        link((self.graph, "load_file"), (loadfile, "value"))
        filename = Text(description = "File to load")
        link((self.graph, "file_name"), (filename, "value"))
        filebox = VBox(children=[filename,loadbutton])
        link((self.graph, "load_file"), (filebox, "visible"))
        

        return ControlPanel(title="Load Dataset", children=[HBox(children=[loaddata,loadfile]),tsbox,filebox])

    def _1dplot(self):
        # create draw mode controls.  NOTE: should only be called once.
        cbar = Checkbox(description="Colorbar")
        link((self.graph, "colorbar"), (cbar, "value"))
        
        autoupdate = Checkbox(description="Auto Update")
        link((self.graph, "autoupdate"), (autoupdate, "value"))
        
        
        cmap = Dropdown(description="Colormap",values=self.graph.COLORMAPS)
        link((self.graph,"colormap"),(cmap,"value"))
             
        color = Dropdown(description="Color",values=self.graph.COLORS.values())
        link((self.graph, "plot_color"), (color, "value"))
        
        kind = Dropdown(description="Plot type", values=PLOTPARSER.keys())
        link((self.graph, "kind"), (kind, "value"))

             
        return ControlPanel(title="1D Plot", 
                children=[
                    cbar,
                    autoupdate,
                    cmap,
                    color,
                    kind
                ]
        )

    def _units(self):
        # create spectrum controls.  NOTE: should only be called once.
        specunit = Dropdown(description="Spectunit",values=self.graph.SPECUNITS.values())
        link((self.graph,"spec_unit"),(specunit,"value"))
        
        varunit = Dropdown(description="Varunit",values=self.graph.VARUNITS.values())
        link((self.graph,"var_unit"),(varunit,"value"))
        
        normunit = Dropdown(description="Normunit",values=self.graph.NORMUNITS.values())
        link((self.graph,"norm_unit"),(normunit,"value"))
        
        return ControlPanel(title="Units",
            children= [
                specunit,
                varunit,
                normunit
            ]
        )
    
    def _sampling(self):
        # create spectrum controls.  NOTE: should only be called once.
        
        sampling_1 = Checkbox(description="By the Axis")
        link((self.graph, "sample_axis"), (sampling_1, "value"))        
        sampling_2 = Checkbox(description="By the Range")
        link((self.graph, "sample_range"), (sampling_2, "value"))
        
        axisrows_1 = IntText(description="Axis Rows",value="0")
        link((self.graph,"axis_rows_1"),(axisrows_1,"value"))        
        axisrows_2 = IntText(description=":",value="100")
        link((self.graph,"axis_rows_2"),(axisrows_2,"value"))        
        axiscols_1 = IntText(description="Axis Columns",value = "0")
        link((self.graph,"axis_cols_1"),(axiscols_1,"value"))        
        axiscols_2 = IntText(description=":",value = "10")
        link((self.graph,"axis_cols_2"),(axiscols_2,"value"))        
        axes = VBox(children=[
                         HBox(children=[axisrows_1, axisrows_2]),
                         HBox(children=[axiscols_1, axiscols_2])])
        link((self.graph,"sample_axis"),(axes,"visible"))

        
        start = FloatSlider(description="Slicing Start Value",value=self.graph.ts.index[0],
                            min=self.graph.ts.index.min(),
                            max=self.graph.ts.index.max()
                            )
        link((self.graph,"slicing_start"),(start,"value"))        
        end = FloatSlider(description="Slicing End Value",value=self.graph.ts.index[-1],
                           min=self.graph.ts.index.min(),
                            max=self.graph.ts.index.max()
                            )
        link((self.graph,"slicing_end"),(end,"value")) 
        spacing = IntText(description="Spacing",value=1)
        link((self.graph,"dataset_spacing"),(spacing,"value"))
        ranges = VBox(children=[start,end,spacing])
        link((self.graph,"sample_range"),(ranges,"visible"))

        
        return ControlPanel(title="Sampling",
              children=[
                  HBox(children=[sampling_1,sampling_2]),
                  axes,
                  ranges
                #avgstyle,
                  ]
              )
    
    def _toolbars(self):
        
        saveplot = Button(description='Save Plot')
        
        savets = Button(description='Save New TS')
        #link((self.graph,"save_ts"),(savets,'value'))
        savets.on_click(lambda x: self.graph.save_to_ns())
        savetsas = Text(description='Save TS as:')
        link((self.graph,"save_ts_as"),(savetsas,'value'))
        #redraw = Button(description="Redraw")
        #redraw.on_click(lambda x: self.graph.draw())
        return ControlPanel(title='Toolbars',children=[saveplot,savets,savetsas])
        #return ToolBar(redraw)      

In [324]:
gui = Spectroscopy(graph=Spectrogram())
gui

in default var
in slicing end
im defaulting to self.ts.index[0]


SpecIndexError: 430.1 is greater than Index max value of 6.797e-07

In [325]:
ts.index.max()

679.70000000000005